In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras import models
from keras import layers
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras import regularizers


# Data Loading

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
train.head()

# Data Preprocessing

In [ ]:
train_labels = train['label']
train1 = train.drop('label', axis=1)

train1 = np.asarray(train1).astype('float32')/255
test = np.asarray(test).astype('float32')/255
train1 = train1.reshape(train1.shape[0], 28, 28, 1)
test = test.reshape(test.shape[0], 28, 28, 1)

train_labels = to_categorical(train_labels)

In [ ]:
img = train1[0]*255
img_reshape = img.reshape(28, 28)
plt.imshow(img_reshape)
plt.show()

In [ ]:
print(train1.shape)
print(train_labels.shape)

In [ ]:
X_train, X_val, y_train, y_val= train_test_split(train1, train_labels, test_size=0.2)

# CNN Model

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), 
                        activation='relu', 
                        input_shape=(28, 28, 1), 
                        padding='same',
                        kernel_regularizer=regularizers.l2(0.001)))

model.add(layers.MaxPooling2D((2, 2), strides=(2,2)))

model.add(layers.Conv2D(32, (3, 3), 
                        activation='relu', 
                        padding='same',
                        kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2)))

model.add(layers.Conv2D(32, (3, 3), 
                        activation='relu', 
                        padding='same',
                        kernel_regularizer=regularizers.l2(0.001)))

model.add(layers.Flatten())

model.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dense(10, activation='softmax'))

from keras import optimizers

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
model.fit(X_train, y_train, 
                    batch_size=128, 
                    epochs=20, 
                    validation_data=(X_val, y_val))

# Model Evaluation

In [ ]:
loss=pd.DataFrame(model.history.history)
loss[['acc', 'val_acc']].plot()

In [ ]:
loss[['loss', 'val_loss']].plot()

In [ ]:
model.fit(train1, train_labels, 
                    batch_size=128, 
                    epochs=20)

# Prediction

In [ ]:
predictions = model.predict(test, batch_size=32)
image_id = range(1, predictions.shape[0] + 1)
pred = [np.argmax(i) for i in predictions]
submission = pd.DataFrame({'ImageId': image_id, 'Label': pred})
submission.to_csv('digit_recognizer_submission', index=False)

In [ ]:
submission.head()

In [ ]:
ss=pd.read_csv('../input/digit-recognizer/sample_submission.csv')
ss